In [11]:
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.preprocessing import image

import matplotlib.pyplot as plt
%matplotlib inline

import models.simpleConv as simpleConv


dataPath = 'RealImageNet/ImageNetSubsample/Data/CLS-LOC'
trainPath = os.path.join(dataPath, 'train')
# testPath = os.path.join(dataPath, 'test')
# validationPath = os.path.join(dataPath, 'val')

img_width, img_height = 224, 224

In [12]:
# from keras.preprocessing import image
# get training data
x_train = []
y_train = []
trainingFolders = [x for x in os.listdir(trainPath)]

lines = [line.rstrip('\n').split() for line in open('RealImageNet/LOC_synset_mapping.txt')]
wnids_to_words = {line[0]:' '.join(line[1:]) for line in lines }

train_bboxes = {} #{image_name: list(bboxes)}
for id in trainingFolders:
    boxesPath = os.path.join("RealImageNet", "LOC_train_solution.csv")
    lines = [line.rstrip('\n').split(',') for line in open(boxesPath)][1:]
    for line in lines:
        imageId = line[0]
        predictionString = line[1]
        split = predictionString.split()
        train_bboxes[imageId] = []
        for i in range(0, len(split), 5):
            box = split[i:i+5]
            train_bboxes[imageId].append(box)

for imageName in train_bboxes.keys():
    imageLabel = imageName.split('_')[0]
    imagePath = os.path.join(trainPath, imageLabel, imageName + ".JPEG") #folder name
    
    if os.path.exists(imagePath):
        img = image.load_img(imagePath, target_size=(224, 224)) #pil format
        x = image.img_to_array(img) 
#         x = np.expand_dims(x, axis=0) # making a numpy array (surrounds with another lis)
#         x_train.append(x/255.)
        for box in train_bboxes[imageName]:
            x_train.append(x/255.)
            idk = [imageLabel[1:]]
            y_train.append(idk)




x_train = np.array(x_train)
# x_train = np.expand_dims(x_train, axis= 1)
y_train = np.array(y_train, dtype = 'uint')
# y_train = np.expand_dims(y_train, axis= 0)


from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

label_encoder = LabelBinarizer()
y_train = label_encoder.fit_transform(y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)



In [13]:
y_train

array([[0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [23]:
model = simpleConv.build_model(img_width, img_height, 3)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=6, verbose=1)

Epoch 1/6
2358/2358 [==============================] - 6s 2ms/step - loss: 1.0300 - acc: 0.5318A: 0s - loss: 1.0355 - acc: 0.
Epoch 2/6
2358/2358 [==============================] - 5s 2ms/step - loss: 0.8585 - acc: 0.6209A: 0s - loss: 0.8604 - acc: 0.6
Epoch 3/6
2358/2358 [==============================] - 5s 2ms/step - loss: 0.7475 - acc: 0.6692
Epoch 4/6
2358/2358 [==============================] - 5s 2ms/step - loss: 0.6790 - acc: 0.6964
Epoch 5/6
2358/2358 [==============================] - 5s 2ms/step - loss: 0.6205 - acc: 0.7277
Epoch 6/6
2358/2358 [==============================] - 5s 2ms/step - loss: 0.5321 - acc: 0.7718


In [24]:
predictedModel = model.predict(x_val)
predictedModel2 = model.predict(x_train)

import pandas as pd
from sklearn.metrics import accuracy_score

dataframe = pd.DataFrame({'pred': label_encoder.inverse_transform(predictedModel), 'true': label_encoder.inverse_transform(y_val)})

dataframe2 = pd.DataFrame({'pred': label_encoder.inverse_transform(predictedModel2), 'true': label_encoder.inverse_transform(y_train)})

print(accuracy_score(dataframe.pred, dataframe.true))
accuracy_score(dataframe2.pred, dataframe2.true)


0.6915254237288135


0.7888040712468194